In [ ]:
import fnmatch
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import seaborn as sns

%matplotlib inline

In [ ]:
# font styles
font_family = "Times"
fs_title = 20
fs_label = 18
fs_small = 14

# init mpl plot style
sns.set_style('ticks')
mpl.rcParams["font.family"] = font_family
mpl.rcParams["mathtext.fontset"] = "stix"
fig_height = 6

In [ ]:
save_figures = True

transformer_train_ids = {
    "2x5": "iter100_lr0.001_wd0.0_bs100_dropout0.1_samplestruct2_lrschedulewarmup_cosine13102022-110204",
    "2x6": "iter100_lr0.001_wd0.0_bs100_dropout0.1_samplestruct2_lrschedulewarmup_cosine13102022-110404",
    "2x7": "iter100_lr0.001_wd0.0_bs100_dropout0.1_samplestruct2_lrschedulewarmup_cosine13102022-110604",
    "2x8": "iter100_lr0.001_wd0.0_bs100_dropout0.1_samplestruct2_lrschedulewarmup_cosine13102022-110704",
    "2x9": "iter100_lr0.001_wd0.0_bs100_dropout0.1_samplestruct2_lrschedulewarmup_cosine13102022-110815",
}

kernel_names = {
    'rbf-kernel': 'RBF Kernel',
}

transformer_model_id = 'gcn_proj_3_16-transformer_l4_d128_h4_featone_hot'

In [ ]:
transformer_res_dir_pattern = '../logs/2d_heisenberg_checkpoints/conditional_heisenberg_{rows}x{cols}/{model_id}/ns{ns}/{train_id}/'
kernel_res_dir_pattern = '../logs/2d_heisenberg_checkpoints/conditional_heisenberg_{rows}x{cols}/{model_id}/ns{ns}/results'

In [ ]:
def get_entropy_error_distribution(props_dir, data_dir, rows, cols, corr_mat_name, model_name):
    qubits = rows * cols
    
    # get hamiltonian ids
    ids = [int(fp[(fp.find('id') + 2):fp.find('.npy')]) for fp in os.listdir(data_dir) if fnmatch.fnmatch(fp, 'entanglement_entropies*.npy')]
    
    # compute rmses
    errors = np.zeros(shape=(qubits, qubits))
    n = 0
    for i in ids:
        cmat_pred = np.load(os.path.join(props_dir, corr_mat_name.format(idx=i)))
        cmat_true = np.load(os.path.join(data_dir, f'entanglement_entropies_id{i}.npy'))
        
        errors = errors + (cmat_pred - cmat_true) ** 2
        n += 1

    rmses = np.sqrt(errors / n)
    np.fill_diagonal(rmses, np.nan)
    rmses = np.reshape(rmses, newshape=(-1))
    
    df = pd.DataFrame.from_dict(data={
        'rmse': rmses,
        'system_size': [f'{rows}x{cols}'] * len(rmses),
        'Model': [model_name] * len(rmses)
    })
    
    return df

In [ ]:
def get_entropy_rmse_df(kernel_id, split, systems_sizes):
    df = None
    
    for (rows, cols, nsamples) in systems_sizes:        
        train_id = transformer_train_ids[f"{rows}x{cols}"]
        res_dir = transformer_res_dir_pattern.format(rows=rows, cols=cols, model_id=transformer_model_id, ns=nsamples, train_id=train_id)
        data_dir = os.path.join(res_dir, 'data', f'{rows}x{cols}', split)
        
        # load transformer errors
        props_dir = os.path.join(res_dir, 'properties', split, 'model', 'entropies')
        dfm = get_entropy_error_distribution(
            props_dir=props_dir, data_dir=data_dir, rows=rows, cols=cols,  corr_mat_name='entropies_model_id{idx}.npy',
            model_name='Transformer (Ours)'
        )

        # load shadow errors
        props_dir = os.path.join(res_dir, 'properties', split, 'shadow', 'entropies')
        dfs = get_entropy_error_distribution(
            props_dir=props_dir, data_dir=data_dir, rows=rows, cols=cols, corr_mat_name='entropies_shadow_id{idx}.npy',
            model_name='Shadow'
        )
        
        # load kernel errors
        res_dir = kernel_res_dir_pattern.format(rows=rows, cols=cols, model_id=kernel_id, ns=nsamples)
        props_dir = os.path.join(res_dir, 'properties', split, 'model', 'entropies')
        data_dir = os.path.join(res_dir, 'data', f'{rows}x{cols}', split)
        dfk = get_entropy_error_distribution(
            props_dir=props_dir, data_dir=data_dir, rows=rows, cols=cols, corr_mat_name='entropies_model_id{idx}.npy',
            model_name=kernel_names[kernel_id]
        )
        
        dfsz = pd.concat([dfm, dfk, dfs], ignore_index=True,axis=0)

        if df is None:
            df = dfsz
            continue

        df = pd.concat([df, dfsz], ignore_index=True,axis=0)
    
    return df

In [ ]:
def make_stripplot(df: pd.DataFrame, title, figsize=(4, 6), save_as=None, y_title=0.98):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    xmax = df.rmse.max()
    
    # make test strips
    sns.stripplot(x="rmse", y="system_size", hue="Model", data=df, alpha=0.5, size=2, dodge=True, ax=ax)
    ax.set_ylabel(None)
    ax.set_xlabel('RMSE', fontsize=fs_label)
    ax.set_xlim(-0.01, xmax+0.05)
    ax.tick_params(labelsize=fs_small)
    
    for _,s in ax.spines.items():
        s.set_linewidth(1)
        s.set_color('black')
    
    ax.legend(fontsize=fs_small, handletextpad=0.1, labelspacing=.1, framealpha=0.0,
              handlelength=0.9, fancybox=False, columnspacing=1, frameon=False, ncol=3, loc='upper center', bbox_to_anchor=(0.5, 1.1))
    
    if title is not None:
        fig.suptitle(title, fontsize=fs_title, y=y_title)
    
    if save_as is not None:
        folder, fn = os.path.split(save_as)
        
        if not os.path.exists(folder):
            os.makedirs(folder)
            
        plt.savefig(save_as, bbox_inches='tight', pad_inches=0.01, dpi=200)
        plt.close(fig)

In [ ]:
def make_stripplot2(df_train: pd.DataFrame, df_test: pd.DataFrame, title, figsize=(7, 4), save_as=None, y_title=0.98):
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    xmax = max(df_train.rmse.max(), df_test.rmse.max())
    
    # make test strips
    ax = axes[0]
    sns.stripplot(x="rmse", y="system_size", hue="Model", data=df_test, alpha=0.5, size=2, dodge=True, ax=ax)
    ax.set_ylabel(None)
    ax.set_xlabel('Test RMSE', fontsize=fs_label)
    ax.set_xlim(-0.01, xmax)
    ax.tick_params(labelsize=fs_small)
    
    for _,s in ax.spines.items():
        s.set_linewidth(1)
        s.set_color('black')
    
    # make train strips
    ax = axes[1]
    sns.stripplot(x="rmse", y="system_size", hue="Model", data=df_train, alpha=0.5, size=2, dodge=True, ax=ax)
    ax.set_ylabel(None)
    ax.set_xlabel('Train RMSE', fontsize=fs_label)
    ax.set_xlim(-0.01, xmax)
    ax.tick_params(labelsize=fs_small)
    
    for _,s in ax.spines.items():
        s.set_linewidth(1)
        s.set_color('black')
    
    axes[0].legend([],[], frameon=False)
    axes[1].legend([],[], frameon=False)
    
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', fontsize=fs_small, handletextpad=0.3, labelspacing=.1,
               handlelength=0.9, columnspacing=1, fancybox=False, frameon=False, framealpha=0.0, ncol=3, bbox_to_anchor=(0.5, 0.99))
    
    plt.subplots_adjust(wspace=0.25, hspace=0.02)
    
    if save_as is not None:
        folder, fn = os.path.split(save_as)
        
        if not os.path.exists(folder):
            os.makedirs(folder)
            
        plt.savefig(save_as, bbox_inches='tight', pad_inches=0.1, dpi=200)
        plt.close(fig)
        
    if title is not None:
        fig.suptitle(title, fontsize=fs_title, y=y_title)

In [ ]:
systems_sizes = [
    (2, 5, 1000),
    (2, 6, 1000),
    (2, 7, 1000),
    (2, 8, 1000),
    (2, 9, 1000),
]

In [ ]:
df = get_entropy_rmse_df('rbf-kernel', 'test', systems_sizes=systems_sizes)

save_as = None
if save_figures:
    save_as = './figures/2d_heisenberg/entropies_kernel_comparison_test.pdf'

make_stripplot(df, title=None, save_as=save_as, y_title=0.99)

In [ ]:
df = get_entropy_rmse_df('rbf-kernel', 'train', systems_sizes=systems_sizes)

save_as = None
if save_figures:
    save_as = './figures/2d_heisenberg/entropies_kernel_comparison_train.pdf'

title = None
make_stripplot(df, title=None, save_as=save_as, y_title=0.99)

In [ ]:
df_test = get_entropy_rmse_df('rbf-kernel', 'test', systems_sizes=systems_sizes)
df_train = get_entropy_rmse_df('rbf-kernel', 'train', systems_sizes=systems_sizes)

save_as = None
if save_figures:
    save_as = './figures/2d_heisenberg/entropies_kernel_comparison.pdf'

title = None
make_stripplot2(df_train, df_test, title=None, save_as=save_as, y_title=0.99)